<a href="https://colab.research.google.com/github/quantaosun/notebook/blob/main/%E6%89%80%E6%9C%89%E4%BA%BA%E5%8F%AF%E7%94%A8%E7%9A%84%E5%88%86%E5%AD%90%E5%AF%B9%E6%8E%A5_Non_covalent_docking_for_everyone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

分子对接是最基本甚至也是最重要的模拟手段之一，许多对接软件需要付费使用，免费的也有，但是眼花缭乱

1.   的选择让人特别是初学者，或者跨学科科研人员难以下手。本脚本旨在为没有付费软件或者初入该领域的跨学科人员提供一个免费在线的对接流程，基本的流程我尽量做的简洁。其中不乏我借鉴的别人的想法或者代码，具体的引用请见最后的说明部分。整个流程整合起来仍是需要许多个小时的工作量，如果您感觉对您有所启发，请给我点击一个喜欢，或者复制这个项目到你自己的账号中。 
Molecular docking is the simplest yet one of the most powerful method of modelling in medicinal chemistry field. There are many licensed and free academic options there, but most still need a local installation. This is a cloud pipline based on Autodock Vina (Smina), aiming to provide to Chinese and Western researchers who want to use molecular docking in their project, for free. Two cloud platform are designed to use are Chinese Baidu AI Studio, and Google Colab. If you like this work, maybe give me a githup like or Baidu AI studio like, thanks.
References and my contributions has been listed at the end of this script
2.  本脚本仅限于在百度AI Studio和谷歌google colab范围内使用，如果你要分享给其他人，请不要删除quantaosun@gmail.com，这可以鼓励我继续提供免费开源的工具，同时被分享者也有可能继续获得本流程的升级版本。 Please keep my email here if you wish, this will encourge me to produce more free tools, also this is an ongoing script, people with my email attatched could get my latest version if they write me an email.


In [ ]:
#@title 准备一些东西，稍等 （Prepare something）
!mkdir external-libraries
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /home/aistudio/external-libraries
#**3.使用conda安装化学信息学软件rdkit**
! /home/aistudio/external-libraries/bin/conda install -c rdkit rdkit -y
#**4.用conda安装格式转换工具open babel**
!/home/aistudio/external-libraries/bin/conda install -q -y -c openbabel openbabel
#**5.使用pip安装可视化工具py3Dmol**
! /home/aistudio/external-libraries/bin/pip install py3Dmol # 3D Molecular Visualizer

In [ ]:
#@title 导入刚才准备的东西 (Import something)
import sys
sys.path.append('/home/aistudio/external-libraries/lib/python3.7/')
#**7.雪对风**
import sys
sys.path.append('/home/aistudio/external-libraries/lib/python3.7/site-packages/')
#**8.大陆对长空（导入所有需要的工具）**
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(rdBase.rdkitVersion)


In [ ]:
#@title 准备对接软件 (Download visualization software)

!wget --no-check-certificate https://sourceforge.net/projects/smina/files/smina.static/download -O smina.static
!chmod u+x smina.static
#**10.Trust but check （验证一下软件状态正常，正常情况下应当打印出使用说明）**
!./smina.static -h
#**11. 下载一个软件pymol**
!yes | /home/aistudio/external-libraries/bin/conda install -c conda-forge pymol-open-source 

# 请修改下面网址链接中的最后四个字母，不要删掉pdb后缀 (change the last four PDB letter, keep the .pdb)

In [6]:
!wget https://files.rcsb.org/download/3htb.pdb

--2021-10-05 01:51:40--  https://files.rcsb.org/download/.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.70
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.70|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-10-05 01:51:41 ERROR 404: Not Found.



# 请自行查找蛋白共晶中包含的小分子的三字母缩写，一般在RCSB 蛋白数据库网页即可看到，替换下文中中的567 (Visit RCSB to find the Three-letter ligand code, substitute 567 with it)
# *使用共晶的四字母缩写，替换1234* (Use the FOUR-letter pdb code to substitute 1234 below)

In [ ]:
#create ligand= resn H3E 
#create active_water= resn HOH within 5 of ligand
#remove resn HOH &! active_water
com_file = open('fetch_and_clean.pml','w')
com_file.write('''
load 1234.pdb
remove resn HOH
h_add elem O or elem N
select 1234-567, resn 567 #Create a selection called 1OYT-FSN from the ligand
select 1234-receptor, 1234 and not 1234-567 #Select all that is not the ligand
save 1234-567.pdb, 1234-567
save 1234-receptor.pdb, 1234-receptor    
''')
com_file.close()

In [ ]:
#@title 预处理靶点 (target preparation before docking)

%cd /content/
! /home/aistudio/external-libraries/bin/pymol -c fetch_and_clean.pml

In [ ]:
#@title 验证对接 (Self dock begin)


!./smina.static -r 6NZP-receptor.pdbqt -l 1234-567.pdbqt --autobox_ligand 1234-LB7.pdbqt --autobox_add 8 --exhaustiveness 16 -o 1234-redock.pdbqt

In [ ]:
#@title 准备一些东西 (Import visualization software)

import py3Dmol

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    #complex_mol=Chem.CombineMols(receptor,mols[-1])
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    #view.addModel(Chem.MolToMolBlock(mols[0]),'sdf')
    chA = {'chain':['H','L','I']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
    #view.setStyle(chA,{'lines': {}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   


In [ ]:
#@title 导入一个小插件 (Import something)

from rdkit import Chem

# 对接后处理，格式转换,更换四字母编码 （Post-dock processing, convert file format for visualization puroose，modify 4-letter code accordingly）

In [ ]:
!/home/aistudio/external-libraries/bin/obabel 3HTB-redock.pdbqt -O 3HTB-redocked.sdf

In [ ]:
#@title 对接后小分子自己做图 (Show image of ligand only)
pose = "0" #@param ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
mols = [m for m in Chem.SDMolSupplier('') if m is not None]
drawit2(mols['pose'])

In [ ]:
#@title 对接后蛋白-小分子做图 (show image of ligand-protien)
pose = "0" #@param ["0", "1", "2", "3", "4", "5", "6", "7", "8"]

receptor = Chem.MolFromPDBFile('3HTB-receptor.pdb')
DrawComplex(receptor,mols[8])

---------------------------**分割线，以上为对接精度验证，对接的是蛋白共晶原有的小分子；以下为对接未知小分子的步骤**---------------------------------------

In [ ]:
#@title 对接前的一些准备 (Import something)

import sys
from collections import defaultdict

import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
import pandas as pd

%pylab inline
IPythonConsole.ipython_3d = True

import py3Dmol
import matplotlib.pyplot as plt
import subprocess

In [ ]:
#@title 请确认目前的文件路径为/content/，上传你的靶点和小分子文件 (upload your protein and ligand please)
%cd /content/
!pwd

In [ ]:
#@title 输入你上传的小分子文件名字，含后缀sdf. Input your name of uploaded sdf file of small molecules, and provide the file name you want to use as a conformasion file generated fromt uploaded file.
sdfFilePath = "" #@param {type:"string"}
ConfoutputFilePath = "" #@param {type:"string"}
#sdfFilePath = 'fxa_ligands.sdf' # The input file of structures to generate conformations from
#ConfoutputFilePath = 'fxa_ligandsForDocking.sdf' # Output file containing conformations for docking
inputMols = [x for x in Chem.SDMolSupplier(sdfFilePath,removeHs=False)]
# Assign atomic chirality based on the structures:
len(inputMols) # Check how many strucures

In [ ]:
#@title 验证是否有一张分子图生成，该分子与项目无关，仅仅为了验证一个对接所需的插件是否正常 (Rdkit validation, the output image has nothign to do with docking itself, just for validation)

mol = Chem.MolFromSmiles('C[NH+]1CCN(CC1)C(=O)Nc1ccc(F)cc1') #This is the ligand in the crystal structure
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
AllChem.MMFFOptimizeMolecule(mol)
mol

In [ ]:
#@title 检测你上传的SDF文件是否合适，这取决于你上传的SDF文件的格式是否恰当(Test if each line of your sdf has a name)

for i, mol in enumerate(inputMols):
    if mol is None:
        print('Warning: Failed to read molecule %s in %s' % (i, sdfFilePath))
    if not mol.GetProp('_Name'):

        print('Warning: No name for molecule %s in %s' % (i, sdfFilePath))

In [ ]:
#@title 准备一点东西(prepare something)

import multiprocessing
from concurrent import futures

In [ ]:
#@title 准备一个小东西 (prepare something)

!/home/aistudio/external-libraries/bin/pip install progressbar

In [ ]:
#@title 小分子构象生成 (Small molecules conformation generation)

import multiprocessing

# Download this from http://pypi.python.org/pypi/futures
from concurrent import futures

# conda install progressbar
import progressbar

#Find number cores available, leave two or system might be unresponsive
numcores = multiprocessing.cpu_count()
max_workers = numcores #-1

#Knowledge based torsion generator http://pubs.acs.org/doi/abs/10.1021/acs.jcim.5b00654
# This function is called in the subprocess.
# The parameters (molecule and number of conformers) are passed via a Python

ps = AllChem.ETKDG()
ps.pruneRmsThresh=0.5
ps.numThreads=0
#Edit for number of confs desired eg n = 5
n=5
def generateconformations(m, n, name):
    m = Chem.AddHs(m)
    ids=AllChem.EmbedMultipleConfs(m, n, ps)
    for id in ids:
        AllChem.UFFOptimizeMolecule(m, confId=id)
    # EmbedMultipleConfs returns a Boost-wrapped type which
    # cannot be pickled. Convert it to a Python list, which can.
    return m, list(ids), name

smi_input_file, sdf_output_file = sys.argv[1:3]


writer = Chem.SDWriter(ConfoutputFilePath)
# suppl = [x for x in Chem.SDMolSupplier(sdfFilePath,removeHs=False)]
#suppl = Chem.SmilesMolSupplier(smi_input_file, titleLine=False)

# for mol in suppl:
#     print(mol.GetPropsAsDict(includePrivate=True).get('_Name'))

with futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
    # Submit a set of asynchronous jobs
    jobs = []
    for mol in inputMols:
        if mol:
            name = mol.GetProp('_Name')
            job = executor.submit(generateconformations, mol, n, name)
            jobs.append(job)

    widgets = ["Generating conformations; ", progressbar.Percentage(), " ",
               progressbar.ETA(), " ", progressbar.Bar()]
    pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(jobs))
    for job in pbar(futures.as_completed(jobs)):
        mol, ids, name = job.result()
        mol.SetProp('_Name', name)
        for id in ids:
            writer.write(mol, confId=id)
writer.close()

In [2]:
#@title 对接开始，请输入四个对接文件名称，一个为靶点相关，三个为小分子相关 (Input one protein related name, three ligand related names)
ProteinForDocking = "" #@param {type:"string"}
LigandFromProtein = "" #@param {type:"string"}
DockedFilePath = "" #@param {type:"string"}
FlexibleDockedFilePath = "" #@param {type:"string"}
#ProteinForDocking = '1f0r_minimized.pdb'

#LigandFromProtein = '1f0r_native_ligand.pdb'
#DockedFilePath = '1f0r_All_Docked.sdf.gz'
#FlexibleDockedFilePath = '1f0r_FlexDocked.sdf.gz'
#**I：对接开始，每次更换蛋白和小分子，只需改上面的变量赋值即可**
!'./smina.static' --cpu 2 --seed 0 --autobox_ligand '{LigandFromProtein}' -r '{ProteinForDocking}' -l '{ConfoutputFilePath}' -o '{DockedFilePath}'

/bin/bash: ./smina.static: No such file or directory


# **对接完成后，请下载后缀为All_Docked_sdf.gz 压缩包到本地电脑，使用其他可视化软件进行对接后的分析即可** (Upon finish, download the All_Docked_sdf.gz to your local laptop)


# **说明：** (References and contribution eleboration)
**1. # This code was first inspired by [https://www.macinchem.org/reviews/JupyterDocking/jupyterdocking.php](http://)
and [https://www.cheminformania.com/ligand-docking-with-smina/](http://)
1. The above two links were written for Google Colab or to be used locally. quantaosun@gmail.com decided to modify it to be used on the Chinese platform Baidu AI studio. AI studio has more strict user permission authorization compared to Colab, so each line of code had been carefully checked before you could see this.
1. the following was changed:
1. **1. The conda installation path was changed from /usr/local to /home/aistudio/external-libraries, to get rid of the incompatible issue with AI studio's python environment. So, all the import sys code had been modified accordingly.
1. 2.Pymol installation command was changed to !/home/aistudio/external-libraries/bin/conda install -c conda-forge pymol-open-source 
1. 3.The docking validation and docking separately existed in two non-realated notebooks, but this one integrated them in a single notebook, which will save time and be more convenient.
1. 4.This notebook realized the "permanent" installation of this docking software, unlike the google colab in which you have to install each time before you could do anything. you should not run most of the codes again, you should directly get it up and running a docking without a further installation.**
1. quantaosun@gmail.com 根据原有的分散的代码块，将其修改和整合后，用于在中国平台百度人工智能平台使用。以下是具体的修改内容
1. conda安装路径已更改，以避免与AI studio 自带的python 环境的不兼容问题
1. Pymol安装命令已更改，以避免一个环境不兼容问题,也添加了提前预置的yes命令，因为AI Studio的非命令界面没有提供半路输入yes的地方。
1. “对接验证”和“对接”分别存在以上两个没有直接关联的来源的网页链接中，quantaosun@gmail.com将它们集成在一个笔记本中，这样可以减少安装公共library 的次数，使用功能也更完善。
1. 这个代码块还实现了这个Smina对接软件的**"永久"**安装，不像谷歌的colab，你必须每次都要安装一遍。这个版本支持你第一次安装后，第二次您不必再次运行大部分代码，您应该直接跳过绝大部分代码，启动并运行对接。
1. 更改了原先的使用conda安装progressbar的方法，该方法在谷歌可行，在百度不行，变更为使用pip
1. 整个脚本分为对接验证和对接两个部分，对接验证是对接共晶自己的配体，对接是对接其他小分子。
1. 这个脚本可以作为谷歌colab简洁版本代码的基础，即隐藏所有代码行，仅展示需要使用者输入的内容行，将会大大提升可读性。